# Agent Groups

#### Create group of agents who can collaborate on a task
#### Agents can be added to a group, and strategies for engagement can be provided.

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.45.0"

Installed Packages Microsoft.SemanticKernel, 1.45.0 Microsoft.SemanticKernel.Agents.Core, 1.45.0 Microsoft.SemanticKernel.Plugins.OpenApi, 1.45.0

In [ ]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.Agents.Chat;
using System.Threading;

#!import config/Settings.cs 
#pragma warning disable SKEXP0110 
#pragma warning disable SKEXP0001

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

// Define the agent names for use in the function template
const string WriterName = "Writer";
const string ReviewerName = "Reviewer";

// Initialize a Kernel with a chat-completion service
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
Kernel kernel = builder.Build();

// Create the agents
ChatCompletionAgent writerAgent =
    new()
    {
        Name = WriterName,
        Instructions = "Write a proposal for a talk on Semantic Kernel.",
        Kernel = kernel
    };

ChatCompletionAgent reviewerAgent =
    new()
    {
        Name = ReviewerName,
        Instructions = """
                You will review the proposal and provide feedback.
                Following are the conditions for the proposal:
                   1. Proposal should be no more than 50 words.
                   2. Proposal should contain the term 'AI Overload' at least once.                
                If the proposal does not meet the conditions, provide the improvement suggestion and ask for a revision. Provide only one improvement suggestion at a time. 
                Only if the proposal meets all above conditions, say 'Looks good' and approve it. 
            """,

        Kernel = kernel
    };

// Create a chat using the defined selection strategy.
AgentGroupChat chat =
    new(writerAgent, reviewerAgent)
    {
        ExecutionSettings = new() { TerminationStrategy = 
            new ApprovalTerminationStrategy()
            {
                Agents = [reviewerAgent],
                MaximumIterations = 10,
            }
        }
    };

    await foreach (var content in chat.InvokeAsync())
    {
        // Print the chat messages to the console
        Console.WriteLine($"{content.AuthorName} says -> ");
        Console.WriteLine($"{content.Content}");
        Console.WriteLine("*************** END OF MESSAGE ***************\n");
    }

    chat.


   class ApprovalTerminationStrategy : TerminationStrategy
   {       
       protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> messages, CancellationToken cancellationToken)
       {
           return Task.FromResult(messages[^1].Content.Contains("Looks good"));
       }
   }

